# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = os.path.join('..', 'WeatherPy', 'cities_weather.csv')
cities_weather = pd.read_csv(file)
cities_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Chifeng,42.2683,118.9636,24.28,88,0,13.47,CN,1612457243
1,1,Jamestown,42.0970,-79.2353,34.00,80,20,5.75,US,1612457119
2,2,Bluff,-46.6000,168.3333,46.65,82,41,7.83,NZ,1612457245
3,3,Poum,-20.2333,164.0167,79.83,83,12,15.82,NC,1612457247
4,4,Khovd,48.0056,91.6419,20.01,81,99,6.73,MN,1612457248
...,...,...,...,...,...,...,...,...,...,...
543,543,Nuuk,64.1835,-51.7216,10.40,67,1,21.85,GL,1612457881
544,544,Buraidah,26.3260,43.9750,73.40,33,40,4.61,SA,1612457824
545,545,Thinadhoo,0.5333,72.9333,82.65,74,52,8.86,MV,1612457922
546,546,Fukuechō,32.6881,128.8419,48.63,53,0,9.37,JP,1612457923


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Store map data in variables
locations = cities_weather[["Lat", "Lng"]]
humidity = cities_weather["Humidity"]

#Plot map
fig = gmaps.figure(map_type='ROADMAP', center = [0, 0], zoom_level = 1.85)

# Create heat layer
heatmap = gmaps.heatmap_layer(locations, weights = humidity)
fig.add_layer(heatmap)

#Display map
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
cities_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Chifeng,42.2683,118.9636,24.28,88,0,13.47,CN,1612457243
1,1,Jamestown,42.0970,-79.2353,34.00,80,20,5.75,US,1612457119
2,2,Bluff,-46.6000,168.3333,46.65,82,41,7.83,NZ,1612457245
3,3,Poum,-20.2333,164.0167,79.83,83,12,15.82,NC,1612457247
4,4,Khovd,48.0056,91.6419,20.01,81,99,6.73,MN,1612457248
...,...,...,...,...,...,...,...,...,...,...
543,543,Nuuk,64.1835,-51.7216,10.40,67,1,21.85,GL,1612457881
544,544,Buraidah,26.3260,43.9750,73.40,33,40,4.61,SA,1612457824
545,545,Thinadhoo,0.5333,72.9333,82.65,74,52,8.86,MV,1612457922
546,546,Fukuechō,32.6881,128.8419,48.63,53,0,9.37,JP,1612457923


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Define desired weather parameters and drop those outside of parameters
index_names = cities_weather[(cities_weather['Max Temp'] < 70) | (cities_weather['Max Temp'] > 80) | 
                             (cities_weather['Cloudiness'] == 0) | (cities_weather['Wind Speed'] > 5)].index 
ideal_weather = cities_weather.drop(index_names, inplace = False)
ideal_weather.dropna(how='any')
ideal_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,Avarua,-21.2078,-159.7750,75.2,94,20,2.30,CK,1612457250
23,23,Vaini,-21.2000,-175.2000,77.0,100,75,2.30,TO,1612457270
63,63,Tiarei,-17.5333,-149.3333,75.2,88,20,2.30,PF,1612457320
64,64,Salalah,17.0151,54.0924,75.2,69,40,3.44,OM,1612457324
66,66,Padang,-0.9492,100.3543,78.8,89,40,2.30,ID,1612457327


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

hotels = []

for i, row in ideal_weather.iterrows():
    hotel_response = requests.get(f"{url}location={row['Lat']},{row['Lng']}&radius=5000&type=hotel&key={g_key}").json()

    hotels.append(hotel_response['results'][0]['name'])

In [9]:
hotel_df = ideal_weather
hotel_df['Hotel Name'] = hotels
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
6,6,Avarua,-21.2078,-159.7750,75.2,94,20,2.30,CK,1612457250,Avarua District
23,23,Vaini,-21.2000,-175.2000,77.0,100,75,2.30,TO,1612457270,Nukuhetulu
63,63,Tiarei,-17.5333,-149.3333,75.2,88,20,2.30,PF,1612457320,Maha'ena
64,64,Salalah,17.0151,54.0924,75.2,69,40,3.44,OM,1612457324,Salalah
66,66,Padang,-0.9492,100.3543,78.8,89,40,2.30,ID,1612457327,Padang


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
